# Exploration des données

## Initialisation

 ### Installation des librairies

In [ ]:
# Mettre a jour l'environnement virtuel et les requirements.txt
%pip install -r ../requirements.txt 
%pip freeze > ../requirements.txt 

### Importation des librairies

In [2]:
# Kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi # mettre clé API kaggle dans C:\Users\<VotreNom>\.kaggle\kaggle.json

# Global
import os

# Data
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning



## Téléchargement Data

Télécharge les fichiers même s'ils existent déjà

In [ ]:
# Authentification
api = KaggleApi()
api.authenticate()

# URL du dataset à télécharger
dataset_url = 'tawsifurrahman/covid19-radiography-database'

# Chemin du dossier où le dataset sera extrait
destination_dir = os.path.abspath("../data")
dataset_folder = os.path.join(destination_dir, dataset_url.split('/')[-1])
print("Dossier de destination :", dataset_folder)

# Création du dossier de destination principal
os.makedirs(destination_dir, exist_ok=True)

# Vérification de l'existence du dossier du dataset
if os.path.exists(dataset_folder):
    print(f"Le dossier du dataset existe déjà : {dataset_folder}")
    print("Téléchargement ignoré.")
else:
    print("Téléchargement du dataset dans :", dataset_folder)
    api.dataset_download_files(
        dataset_url,
        path=dataset_folder,
        unzip=True
    )
    print("Dataset téléchargé avec succès dans :", dataset_folder)
    

Dossier de destination : c:\Users\Léna\Documents\A_Repos\DS_COVID\data\covid19-radiography-database
Téléchargement du dataset dans : c:\Users\Léna\Documents\A_Repos\DS_COVID\data\covid19-radiography-database
Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database


## Analyse des Data

In [ ]:
normal_metadata = pd.read_excel(os.path.join(path2, 'Normal.metadata.xlsx'), sheet_name='Sheet1')
normal_metadata.head()